In [1]:
!pip install PyPDF2

In [1]:
#Given the whole pdf return the index of the summary page
def get_summary_page(pages):
    summary_page_index = 0

    for i, page in enumerate(reader.pages):
        text = page.extract_text()

        #Using substring "Page 1:" to check if the page is the one with the summary 
        if 'Page 1:' in text:
            summary_page_index = i
            break
    
    return (summary_page_index)

#Given the summary page return the name, the year and the publisher of the game
def get_game_info(page):
    name = re.search('Game:(.*)\n', page).group(1).strip().upper().rstrip(":")
    #Publisher: ....
    if re.search('Publisher:(.*)\n', page):
        #Publisher: publisher (year)
        if re.search('Publisher:(.*)\)\n', page):
            publisher_and_year = re.search('Publisher:(.*)\)\n', page).group(1).strip() +')'
            publisher = re.search('(.*)\(', publisher_and_year).group(1).strip().upper()
            year = re.search('\((.*)\)', publisher_and_year).group(1).strip()
        #Publisher: publisher
        else:
            publisher = re.search('Publisher:(.*)\n', page).group(1).strip().upper()
            year = None
    #Pub: ...
    elif re.search('Pub:(.*)\n', page):
        #Pub: ©year publisher
        if re.search('Pub: ©(.*)\n', re.sub(' +', ' ', page)):
            publisher_and_year = re.search('Pub: ©(.*)\n', re.sub(' +', ' ', page)).group(1).strip()
            publisher = publisher_and_year[5:].upper()
            year = publisher_and_year[:4]
        #Pub: publisher (year)
        elif re.search('Pub:(.*)\)\n', page):
            publisher_and_year = re.search('Pub:(.*)\)\n', page).group(1).strip() +')'
            publisher = re.search('(.*)\(', publisher_and_year).group(1).strip().upper()
            year = re.search('\((.*)\)', publisher_and_year).group(1).strip()
        #Pub: publisher
        else:
            publisher = re.search('Pub:(.*)\n', page).group(1).strip().upper()
            year = None
    #Impossible to retrieve year and publisher
    else:
        publisher = None
        year = None
    return (name, year, publisher)

#From the original pdf return only those pages that contain rules
def get_rules(current_index, pages):
    line_by_line_text = pages[current_index].extract_text().split('\n')
    rules_pages = 0

    for line in line_by_line_text:
        if re.search('Page ([0-9]): Rules', re.sub(' +', ' ', line)):
            rules_pages = int(re.search('Page ([0-9]): Rules', re.sub(' +', ' ', line)).group(1))
            
    return get_text(pages[current_index + 1: current_index + rules_pages + 1])

#From n pages return only one string after having deleted '\n' char
def get_text(pages):
    text = ""
    raw_text = ""
    for page in pages:
        #Substitute '\n' with blank and remove multiple blanks
        text += re.sub(' +', ' ', page.extract_text().replace('\n',' ')).lower()
        raw_text += page.extract_text() #Need to maintain \n for paragraph analysis
    return text, raw_text

#Return game information as a dictionary
def get_dict(game_info, rules, raw_rules):
    return {"name": game_info[0], "year": game_info[1], "publisher": game_info[2], 
                "rules": rules, "raw_rules": raw_rules}

In [2]:
import os
from PyPDF2 import PdfReader
import re

PATH = './EOOG_rulebooks/'
files = os.listdir(PATH)
files.remove('.DS_Store')
files.sort()

list_of_dict = list()

for file in files:
    file_path = PATH + file
    print(file)
    
    reader = PdfReader(file_path)
    
    summary_page = get_summary_page(reader.pages)
    game_info = get_game_info(reader.pages[summary_page].extract_text())
    rules, raw_rules = get_rules(summary_page, reader.pages)
    
    if rules != '':
        list_of_dict.append(get_dict(game_info, rules, raw_rules))

3000Scoundrels_v1.pdf
7Ronin_v1.pdf
7Wonders_v1.2.pdf
AGameofThrones_v2.pdf
AStudyinEmerald_v2.pdf
AT43_v2.4.pdf
ATouchofEvil_v3.3.pdf
AVPTheHuntBegins_v1.1.pdf
AWarofWhispers_v1.1.pdf
Abomination_v1.pdf
AdvSpaceCrusade_v1.1.pdf
AeonTrespassOdyssey_v1.pdf
AgeofConan_v3.2.pdf
Agricola_v1.5.pdf
AlienFateoftheNostromo_v1.pdf
Aliens_v1.3.pdf
AncientTerribleThings_v1.pdf
AndroidNetrunner_v2.1.pdf
Android_v1.3.pdf
Ankh_v1.1.pdf
Arcana_v1.1.pdf
Archaeology_v1.1.pdf
Archmage_v1.pdf
ArcticScavengers_v2.pdf
ArenaMaximus_v3.2.pdf
ArkhamHorrorCardGame_v1.pdf
Armada_v1.2.pdf
ArrakisDawnoftheFremen_v1.pdf
AssassinorumExecutionForce_v1.pdf
AssassinsCreed_v1.1.pdf
AtlantisRising_v1.pdf
AveCaesar_v1.pdf
Aztlan_v1.pdf
Babel_v2.2.pdf
BacktotheFuture_v1.1.pdf
BatmanGothamCityChronicles_v1.2.pdf
BattlKhaos_v1.pdf
BattleLore_v4.1.pdf
BattleMasters_v1.1.pdf
Battleball_v1.pdf
BattleforArmageddon_v1.pdf
BattleshipGalaxies_v1.pdf
BattlestarGalactica_v4.4.pdf
BeowulftheMovie_v1.pdf
BioshockInfinite_v1.2.pdf
Blac

Tokaido_v1.1.pdf
TradersofCarthage_v1.2.pdf
TreasureIsland_v2.pdf
Tusk_v1.pdf
TwilightImperium4thEd_v1.pdf
TyrantsoftheUnderdark_v1.pdf
UlayaChronicles_v1.pdf
UndauntedNormandy_v1.1.pdf
UndauntedNorthAfrica_v1.pdf
VSabotage_v2.pdf
VikingsGoneWild_v2.pdf
VoicesInMyHead_v1.pdf
W40K9thEdition_v1.pdf
W40KCombatArena_v1.pdf
W40KConquest_v1.pdf
W40KHeroesofBlackReach_v1.2.pdf
W40KKillTeam_v1.pdf
WFRP3_v2.pdf
WQBlackstoneFortress_v1.2.pdf
WQSilverTower_v1.2.pdf
Warcry_v2.1.pdf
WarhammerDiskwars_v1.pdf
WarhammerInvasion_v2.pdf
WarhammerQuestCardGame_v1.1.pdf
WarhammerQuestCursedCity_v1.pdf
WarhammerQuestLostRelics_v1.pdf
WarhammerQuest_v1.1.pdf
WarlockFiretopMountain_v1.pdf
Warlock_v1.pdf
WaroftheRing_v2.2.pdf
WaroftheRing_v5.5.pdf
WarriorKnights_v2.pdf
WasteKnights_v1.pdf
WesternLegends_v2.2.pdf
WildWestExodus_v1.pdf
Wildlands_v2.1.pdf
WingsofWar_v3.2.pdf
WiseGuys_v1.pdf
WitchofSalem_v1.1.pdf
WizWar_v2.1.pdf
WoWAG_v1.pdf
WorldofTanks_v1.pdf
WrathofAshardalon_v1.1.pdf
WrathofKingsBW_v2.2.pdf
W

In [5]:
import pandas as pd

pd.DataFrame(list_of_dict).to_csv('dataset_from_pdfs.csv')

In [3]:
import requests
import xml.etree.ElementTree as ET
import difflib
import csv


def get_id_by_name(name):
    search_url = 'https://boardgamegeek.com/xmlapi/search?search=' + name + '&exact=1'
    
    x = requests.get(search_url)
    bgg_id = -1
    
    if(x.status_code == 200):
        parser = ET.XMLPullParser(['start'])
        parser.feed(x.text)

        for event, elem in parser.read_events():
            if(elem.tag == 'boardgame'):
                bgg_id = elem.get('objectid')
    
    if(bgg_id == -1):
        bgg_id = not_exact_search(name)
    
    return bgg_id

def get_weight_by_id(id):
    test_url = "https://boardgamegeek.com/xmlapi2/thing?id=" + id + "&stats=1"

    x = requests.get(test_url)
    weight = 0.0
    
    if(x.status_code == 200):      
        parser = ET.XMLPullParser(['start'])
        parser.feed(x.text)

        for event, elem in parser.read_events():
            if(elem.tag == 'averageweight'):
                weight = elem.get('value')
        
        return weight
        
            
#search by name without the exact attribute. Three possibilities:
# 1. Find nothing and so removing the record from the dataset
# 2. Find only 1 game and so this is the one that I have been searching for
# 3. Find more than 1 game for an entry. Choose between them the most similar
def not_exact_search(name):
    search_url = 'https://boardgamegeek.com/xmlapi/search?search=' + name

    x = requests.get(search_url)
    bgg_id = -1
    s = []
    
    if(x.status_code == 200):
        text = x.text
        
        parser = ET.XMLPullParser(['start'])
        parser.feed(text)
        
        #print(text)
        
        result_count = 0
        
        
        for event, elem in parser.read_events():
            if(elem.tag == 'boardgame'):
                result_count += 1
                #bgg_id = elem.get('objectid')
        
        if(result_count == 1):
            parser = ET.XMLPullParser(['start'])
            parser.feed(text)
            for event, elem in parser.read_events():
                if(elem.tag == 'boardgame'):
                    bgg_id = elem.get('objectid')
        elif(result_count > 1):
            parser = ET.XMLPullParser(['start'])
            parser.feed(text)
            
            possible_games = dict()
            tmp_id = 0
            tmp_name = ""
            
            for event, elem in parser.read_events():
                if(elem.tag == 'boardgame'):
                    tmp_id = elem.get('objectid')
                if(elem.tag == 'name'):
                    tmp_name = elem.text
                    possible_games[tmp_name.lower()] = tmp_id
                     
            s = difflib.get_close_matches(name.lower(), list(possible_games.keys()), 1, 0.7)
            
    if(len(s) > 0):
        return possible_games.get(s[0])
    else:
        return -1

In [54]:
updated_dataset = list()

for d in list_of_dict:
    bbg_id = get_id_by_name(d.get('name'))
    if(bbg_id != -1):
        weight = get_weight_by_id(bbg_id)
        d['weight'] = weight
        d['id'] = bbg_id
        
        #Maintain the record only if the weight is not 0
        if d['weight'] != 0:
            updated_dataset.append(d)

In [74]:
import pandas as pd

df = pd.DataFrame(updated_dataset)
new_cols = ["id", "name", "year", "publisher", "weight", "rules", "raw_rules"]
df = df.drop_duplicates(subset=['name'], keep='first')
df=df.reindex(columns=new_cols)
df

,id,name,year,publisher,weight,rules,raw_rules
0,367525,3000 SCOUNDRELS,2022,UNEXPECTED GAMES,2.1667,"setup before your first game, place all clear ...","SETUP\nBefore your first game, place all clear..."
1,154173,7 RONIN,2013,GREY FOX GAMES,2.2105,setup choose or determine randomly who is the ...,SETUP\nChoose or determine randomly who is the...
2,68448,7 WONDERS,2010,ASMODEE,2.321,"setup depending on the number of players, retu...","Setup\nDepending on the number of players, ret..."
3,141517,A STUDY IN EMERALD,2013,TREEFROG GAMES,3.4538,setup shuffle the secret identity cards and de...,SETUP\nShuffle the Secret Identity cards and d...
4,170953,A TOUCH OF EVIL,2008,FLYING FROG GAMES,0,setup basic game set aside the curse of the we...,SETUP\nBasic game\nSet aside the Curse of the ...
...,...,...,...,...,...,...,...
345,146451,WRATH OF KINGS,2014,COOL MINI OR NOT,2.9,basics distances (in inches) may be measured a...,BASICS\nDistances (in inches) may be measured ...
347,8593,WRECKAGE,2003,FANTASY FLIGHT GAMES,1.7222,setup 1. starting with a randomly chosen first...,Setup\n1. Starting with a randomly chosen firs...
348,38868,ZANZIAR,2009,DRAGON DAWN PRODUCTIONS,0,setup each player takes: 3 pairs of adventurin...,Setup\nEach player takes:\n3 pairs of Adventur...
349,2471,ZOMBIES!!!,2001,"TWILIGHT CREATIONS, INC.",1.6142,setup place the town square tile in the centre...,SETUP\nPlace the Town Square tile in the centr...


In [72]:
df.to_csv("boardgames_dataset_NEW.csv")